# Libbitcoin BX: Payment Channel

In this BX tutorial, we will demonstrate a simple payment channel between two participants, Alice and Bob.

* Wallet Setup for Alice & Bob
* 1) Opening a payment channel
* 2) Updating the payment channel
* 3) Alice broadcasts old state. Bob sweeps the channel.



<hr style="border: 0.5px dashed #000;">


## Wallet setup for Alice & Bob

In [1]:
bx seed | bx ec-new

f8cc264660fe34de75466b1a3173f764b09efb35c5689f74f5bb7da62eb6ae28


In [2]:
bx ec-to-public dbafbb58c0e58a0609898e831d387e7ed63275e21c5755f1b4600b36cb314cbb &&
bx ec-to-public 2d80969c3c10a85ce8a887686f9faad63d0b8bd05f6b5f4e22e425073c263129

02b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5
027546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a


In [3]:
bx sha256 02b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5 | bx ripemd160 &&
bx sha256 027546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a | bx ripemd160

eb8e87d55a0a313628d70603ab342dd7c380993e
8d950d086cf089d0bc29a0184faa7a448fd8c841


In [4]:
bx ec-to-address -v 111 02b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5 &&
bx ec-to-address -v 111 027546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a


n2zTqLKDrSF79wA7dni9mRbGQAP8ifMTbh
mtRa8Liyeygehg3b4JgEowjSCH9DqbR3pX


**Note your spendable UXTOs here:**
```
TXID: 2d8eddc045e5aa39b5d8d35374cf597a35f14a00d6163bfa147da52d4714e1d9
Index: 0
Amount: 65000000
```

<hr style="border: 0.5px dashed #000;">

## 1) Opening the payment channel
<br>
<img src="images/payment_channel_funding.jpg" alt="drawing" style="" width="700px"/>

## Alice creates the funding transaction

### A + B multisig output script
* `2`
* `public key A`
* `public key B`
* `2`
* `checkmultisig`


In [ ]:
bx script-encode \
"2 [04b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc] [047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb2] 2 CHECKMULTISIG"


### Minimum fee calculation
* `100 Satoshi/sigops + 1 Satoshi/Byte` 
* Multisig counts as `4 * 20 sigops`
* Output amount(s) are previous output amount(s) minus fees

In [ ]:
# Output amount
expr 65000000 - 20 \* 4 \* 100 - 340


### Encode funding transaction for signing.

In [ ]:
bx tx-encode \
--input 2d8eddc045e5aa39b5d8d35374cf597a35f14a00d6163bfa147da52d4714e1d9:0 \
--output 524104b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc41047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb252ae:64991660


### Alice signs funding transaction.

In [ ]:
bx fetch-tx -f json 2d8eddc045e5aa39b5d8d35374cf597a35f14a00d6163bfa147da52d4714e1d9 \
| jq ".transaction.outputs[0].script"


In [ ]:
bx input-sign \
dbafbb58c0e58a0609898e831d387e7ed63275e21c5755f1b4600b36cb314cbb \
"dup hash160 [eb8e87d55a0a313628d70603ab342dd7c380993e] equalverify checksig" \
0100000001d9e114472da57d14fa3b16d6004af1357a59cf7453d3d8b539aae545c0dd8e2d0000000000ffffffff01acb1df030000000087524104b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc41047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb252ae00000000


In [ ]:
bx input-set \
"[3045022100855e10a0526742673ff226cdc681f34d075ff13159784d29640eaf6e74a8f486022009ca1a0b3d0361f5a0ac95c7a6a653adf61194667f5ffac57a4f2e8c18d439c501] [02b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5]" \
0100000001d9e114472da57d14fa3b16d6004af1357a59cf7453d3d8b539aae545c0dd8e2d0000000000ffffffff01acb1df030000000087524104b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc41047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb252ae00000000


In [ ]:
bx validate-tx \
0100000001d9e114472da57d14fa3b16d6004af1357a59cf7453d3d8b539aae545c0dd8e2d000000006b483045022100855e10a0526742673ff226cdc681f34d075ff13159784d29640eaf6e74a8f486022009ca1a0b3d0361f5a0ac95c7a6a653adf61194667f5ffac57a4f2e8c18d439c5012102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5ffffffff01acb1df030000000087524104b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc41047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb252ae00000000


In [ ]:
bx tx-decode -f json \
0100000001d9e114472da57d14fa3b16d6004af1357a59cf7453d3d8b539aae545c0dd8e2d000000006b483045022100855e10a0526742673ff226cdc681f34d075ff13159784d29640eaf6e74a8f486022009ca1a0b3d0361f5a0ac95c7a6a653adf61194667f5ffac57a4f2e8c18d439c5012102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5ffffffff01acb1df030000000087524104b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc41047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb252ae00000000 \
| jq ".transaction.hash"


**Important:** Alice can only broadcast the funding transaction once commitment 0 has been signed by both parties, as in the following section. Otherwise, her funds could become unspendable without Bob's cooperation. 

## 1.2) Commitment transactions `0a/0b`


### 1.2.1) Commitment transaction `0a`

#### Output Scripts
* RSMC A
* PKPKH B

#### RSMC A
* `IF`
    * `[public key R]`
* `ELSE`
    * `[3-byte delay]`
    * `checksequenceverify`
    * `drop`
    * `[public key A]`
* `ENDIF`
* `CHECKSIG`

#### `Secret r = r_A + r_B`
* Both Alice and Bob independently generate secrets `r_A` and `r_B`
* Alice and Bob only exchange public keys `R_A` AND `R_B`
* `public key R = R_A + R_B = r_A * G + r_B * G`

In [ ]:
bx ec-to-public fde351a976918c9b03ffca81788792e441b4383fbd08c7b0938232f74eabbdb5 &&
bx ec-to-public a7f98d6d45a96f94f314034535b2e5d86c659adde3e05d3b91b8c1a84b597ffd

#### Alice computes `public key R` by adding `R_B` to` r_A * G:`


In [ ]:
bx ec-add 031111a478bb24bd57e67ee3bacb920f2b38d6730ef9cd98f0386b4eea6eafee7a fde351a976918c9b03ffca81788792e441b4383fbd08c7b0938232f74eabbdb5


#### Encoding Output Scripts for Commitment `0a`

In [ ]:
#RSMC A
bx script-encode \
"IF [0358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed044058] ELSE [050000] CHECKSEQUENCEVERIFY DROP [02b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5] ENDIF CHECKSIG"


In [ ]:
#P2PKH B
bx script-encode \
"DUP HASH160 [8d950d086cf089d0bc29a0184faa7a448fd8c841] EQUALVERIFY CHECKSIG"


#### Minimum Fee Calculation

In [ ]:
# Output Amounts minus fees 
# 2 sigops
expr 64991660 - 2 \* 4 \* 100 - 350

**Encode Commitment `0a` for signing**

In [ ]:
bx tx-encode \
--input 8d2b546e2d4ca3342fff120efba60cf1b11da993f6c90442e3ca44da0e857908:0 \
--output 63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac:64900510 \
--output 76a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac:90000


**Signing by Alice & Bob**

In [ ]:
bx input-sign dbafbb58c0e58a0609898e831d387e7ed63275e21c5755f1b4600b36cb314cbb \
"2 [04b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc] [047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb2] 2 checkmultisig" \
01000000010879850eda44cae34204c9f693a91db1f10ca6fb0e12ff2f34a34c2d6e542b8d0000000000ffffffff029e4dde03000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac905f0100000000001976a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000 \
&&
bx input-sign 2d80969c3c10a85ce8a887686f9faad63d0b8bd05f6b5f4e22e425073c263129 \
"2 [04b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc] [047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb2] 2 checkmultisig" \
01000000010879850eda44cae34204c9f693a91db1f10ca6fb0e12ff2f34a34c2d6e542b8d0000000000ffffffff029e4dde03000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac905f0100000000001976a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000


In [ ]:
bx input-set \
"zero [30440220188d7a4e53c3e7e745bedec01fbe98c40ad08b78a34b55911c0c59541e9f47bf0220625b3aac3e56b5a8100cbbc1a92faa4969c7303a236c8ed688a9731e013bd71c01] [3045022100eca3443f6f2980d32434e6d8f7ea6c56127102bc897bab51d4e5ea66f71b01d102200b4aa7ca6a509cd969ddad25a4718e8817930cba724f0c00ec26d42d254032fc01]" \
01000000010879850eda44cae34204c9f693a91db1f10ca6fb0e12ff2f34a34c2d6e542b8d0000000000ffffffff029e4dde03000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac905f0100000000001976a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000


In [ ]:
bx validate-tx \
01000000010879850eda44cae34204c9f693a91db1f10ca6fb0e12ff2f34a34c2d6e542b8d0000000000ffffffff029e4dde03000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac905f0100000000001976a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000


**With commitment `0a` now complete, Alice is now ready to broadcast the funding transaction**

In [ ]:
bx send-tx \
01000000010879850eda44cae34204c9f693a91db1f10ca6fb0e12ff2f34a34c2d6e542b8d0000000000ffffffff029e4dde03000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac905f0100000000001976a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000


### 1.2.2) Commitment transaction `0b`
* Not shown
* Follows section 1.2.1

<hr style="border: 0.5px dashed #000;">


## 2) Updating the channel
<br>
<img src="images/payment_channel_update.jpg" alt="drawing" style="" width="700px"/>

### 2.1) New commitment transaction `1a`/`1b` are created
* Not shown 
* Follows section 1.2


### 2.2) Alice reveals `r_A` to Bob
* Since Alice is paying Bob, she must revoke her past commitment TX output. 
* Bob can now compute `r` = `r_A + r_B`

In [ ]:
bx ec-add-secrets \
fde351a976918c9b03ffca81788792e441b4383fbd08c7b0938232f74eabbdb5 \
a7f98d6d45a96f94f314034535b2e5d86c659adde3e05d3b91b8c1a84b597ffd


In [ ]:
bx ec-to-public a5dcdf16bc3afc2ff713cdc6ae3a78bdf36af636f1a084b065689612c9cefc71
0358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed044058


<hr style="border: 0.5px dashed #000;">

## 3) Alice broadcasts old state. Bob sweeps the channel.
<br>
<img src="images/payment_channel_penalty.jpg" alt="drawing" style="" width="700px"/>

### 3.1) Alice broadcasts commitment tx `0a`

In [ ]:
bx send-tx 01000000010879850eda44cae34204c9f693a91db1f10ca6fb0e12ff2f34a34c2d6e542b8d0000000092004730440220188d7a4e53c3e7e745bedec01fbe98c40ad08b78a34b55911c0c59541e9f47bf0220625b3aac3e56b5a8100cbbc1a92faa4969c7303a236c8ed688a9731e013bd71c01483045022100eca3443f6f2980d32434e6d8f7ea6c56127102bc897bab51d4e5ea66f71b01d102200b4aa7ca6a509cd969ddad25a4718e8817930cba724f0c00ec26d42d254032fc01ffffffff029e4dde03000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac905f0100000000001976a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000


In [ ]:
bx tx-decode -f json \
01000000010879850eda44cae34204c9f693a91db1f10ca6fb0e12ff2f34a34c2d6e542b8d0000000092004730440220188d7a4e53c3e7e745bedec01fbe98c40ad08b78a34b55911c0c59541e9f47bf0220625b3aac3e56b5a8100cbbc1a92faa4969c7303a236c8ed688a9731e013bd71c01483045022100eca3443f6f2980d32434e6d8f7ea6c56127102bc897bab51d4e5ea66f71b01d102200b4aa7ca6a509cd969ddad25a4718e8817930cba724f0c00ec26d42d254032fc01ffffffff029e4dde03000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac905f0100000000001976a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000 \
| jq ".transaction.hash"


### 3.2) Bob sweeps funds from commitment tx `0a`

#### Encode penalty transaction for signing

In [ ]:
# Encode output spendable by Bob
bx script-encode \
"DUP HASH160 [8d950d086cf089d0bc29a0184faa7a448fd8c841] EQUALVERIFY CHECKSIG"


In [ ]:
expr 64990510 - 4 \* 100 - 350

In [ ]:
bx tx-encode \
--input 796d7d2d3adec17738b67c357e0d6ebd0f808c4bd03c296e85022b517a60a71e:0 \
--input 796d7d2d3adec17738b67c357e0d6ebd0f808c4bd03c296e85022b517a60a71e:1 \
--output 76a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac:64989760


#### Bob signs input to spend RSMC A
* `[Sig R]`
* `1`

In [ ]:
bx fetch-tx -f json 796d7d2d3adec17738b67c357e0d6ebd0f808c4bd03c296e85022b517a60a71e \
| jq ".transaction.outputs[0].script"


In [ ]:
# Bob signs input 0 (RSMC) with secret r.
bx input-sign --index 0 a5dcdf16bc3afc2ff713cdc6ae3a78bdf36af636f1a084b065689612c9cefc71 \
"if [0358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed044058] else [050000] checksequenceverify drop [02b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5] endif checksig" \
01000000021ea7607a512b02856e293cd04b8c800fbd6e0d7e357cb63877c1de3a2d7d6d790000000000ffffffff1ea7607a512b02856e293cd04b8c800fbd6e0d7e357cb63877c1de3a2d7d6d790100000000ffffffff0140aadf03000000001976a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000


#### Bob signs input to spend P2PKH B

In [ ]:
bx fetch-tx -f json 796d7d2d3adec17738b67c357e0d6ebd0f808c4bd03c296e85022b517a60a71e \
| jq ".transaction.outputs[1].script"


In [ ]:
# Bob signs input 1 (p2pkh) with private key.
bx input-sign --index 1 2d80969c3c10a85ce8a887686f9faad63d0b8bd05f6b5f4e22e425073c263129 \
"dup hash160 [8d950d086cf089d0bc29a0184faa7a448fd8c841] equalverify checksig" \
01000000021ea7607a512b02856e293cd04b8c800fbd6e0d7e357cb63877c1de3a2d7d6d790000000000ffffffff1ea7607a512b02856e293cd04b8c800fbd6e0d7e357cb63877c1de3a2d7d6d790100000000ffffffff0140aadf03000000001976a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000


#### Finalise penalty transaction


In [ ]:
bx input-set --index 0 \
"[304402202f79d5255845e05d4e3d3688f191a35c61c647080a56754bad647f28e1dfe14702203ff30d1e22ce7396b2b5f78dee6ea76934c8567f7f0c86f5954fba04256e16db01] 1" \
01000000021ea7607a512b02856e293cd04b8c800fbd6e0d7e357cb63877c1de3a2d7d6d790000000000ffffffff1ea7607a512b02856e293cd04b8c800fbd6e0d7e357cb63877c1de3a2d7d6d790100000000ffffffff0140aadf03000000001976a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000


In [ ]:
bx input-set --index 1 \
"[3045022100e21adb334909a91f49ef53c038823199ec51c6a2a483a7a5ced56990b98aa44202206d2fa178206808bd4be1dbdc3e26934b77f29d7c4f893153e6068c2afac1d0ee01] [027546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a]" \
01000000021ea7607a512b02856e293cd04b8c800fbd6e0d7e357cb63877c1de3a2d7d6d79000000004947304402202f79d5255845e05d4e3d3688f191a35c61c647080a56754bad647f28e1dfe14702203ff30d1e22ce7396b2b5f78dee6ea76934c8567f7f0c86f5954fba04256e16db0151ffffffff1ea7607a512b02856e293cd04b8c800fbd6e0d7e357cb63877c1de3a2d7d6d790100000000ffffffff0140aadf03000000001976a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000


In [ ]:
bx validate-tx 01000000021ea7607a512b02856e293cd04b8c800fbd6e0d7e357cb63877c1de3a2d7d6d79000000004947304402202f79d5255845e05d4e3d3688f191a35c61c647080a56754bad647f28e1dfe14702203ff30d1e22ce7396b2b5f78dee6ea76934c8567f7f0c86f5954fba04256e16db0151ffffffff1ea7607a512b02856e293cd04b8c800fbd6e0d7e357cb63877c1de3a2d7d6d79010000006b483045022100e21adb334909a91f49ef53c038823199ec51c6a2a483a7a5ced56990b98aa44202206d2fa178206808bd4be1dbdc3e26934b77f29d7c4f893153e6068c2afac1d0ee0121027546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2affffffff0140aadf03000000001976a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000
